In [8]:
import json
import pandas as pd
import plotly.express as px

In [9]:
data = pd.read_csv('time_tweet.csv')
data

,User,Text,Sentiment,Time
0,FrDaveNix,RT @yesisworld: If you drive drunk you get a p...,Negative,Fri Jun 10 2022
1,FrDaveNix,https://t.co/RaW0xJuuaJ,Neutral,Tue Jun 14 2022
2,FrDaveNix,RT @JackPosobiec: Jan 6 Committee hearings can...,Neutral,Wed Jun 15 2022
3,Bishop_Sheen,"The proud man counts his newspaper clippings, ...",Positive,Fri Jun 10 2022
4,NovusOrdoWatch,The linchpin keeping this theological madhouse...,Neutral,Fri Jun 10 2022
...,...,...,...,...
225,Bug_Hall,6/\nI will remain saddened by the friends I've...,Negative,Wed Jun 15 2022
226,Bug_Hall,"5/\n... consumerist brandism. In fact, I'd arg...",Neutral,Wed Jun 15 2022
227,Bug_Hall,4/\n... watched as the arbitrary lines drawn h...,Neutral,Wed Jun 15 2022
228,Bug_Hall,"3/\nWhen I came out publicly as a Catholic, I ...",Neutral,Wed Jun 15 2022


In [25]:
gb = data.groupby(['Time',]).count()['User']
gb

Time
Fri Jun 10 2022     30
Tue Jun 14 2022    100
Wed Jun 15 2022    100
Name: User, dtype: int64

In [16]:
gb.to_dict('records')

[{'User': 30, 'Text': 30, 'Sentiment': 30},
 {'User': 100, 'Text': 100, 'Sentiment': 100},
 {'User': 100, 'Text': 100, 'Sentiment': 100}]

In [11]:
neg = data[data['Sentiment'] == 'Negative']
neu = data[data['Sentiment'] == 'Neutral']
pos = data[data['Sentiment'] == 'Positive']

pos.groupby('Time').count().loc['Fri Jun 10 2022']['User']

4

In [22]:
new_data = pd.DataFrame(columns=['Date', 'Sentiment', 'Count'])
for frame, sentiment in zip([neg, neu, pos], ['Negative', 'Neutral', 'Positive']):
    frame = frame.groupby('Time').count()
    for i, date in enumerate(frame.index):
        row = pd.DataFrame([{'Date': date, 'Sentiment': sentiment,
                    'Count': frame.loc[date]['User']}])
        new_data = pd.concat([new_data, row], ignore_index=True)
print(new_data)

print(new_data.groupby('Date').agg('sum'))

              Date Sentiment Count
0  Fri Jun 10 2022  Negative     6
1  Tue Jun 14 2022  Negative    24
2  Wed Jun 15 2022  Negative    28
3  Fri Jun 10 2022   Neutral    20
4  Tue Jun 14 2022   Neutral    52
5  Wed Jun 15 2022   Neutral    52
6  Fri Jun 10 2022  Positive     4
7  Tue Jun 14 2022  Positive    24
8  Wed Jun 15 2022  Positive    20
                               Sentiment  Count
Date                                           
Fri Jun 10 2022  NegativeNeutralPositive     30
Tue Jun 14 2022  NegativeNeutralPositive    100
Wed Jun 15 2022  NegativeNeutralPositive    100


In [13]:
px.line(new_data, x='Date', y='Count', color='Sentiment')

In [14]:
px.bar(new_data, x='Date', y='Count', color='Sentiment')